In [262]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import numpy as np
from bs4 import BeautifulSoup
import requests
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')

from time import sleep
import re

In [382]:
#url = 'https://www.scoresandodds.com/nba?date=2023-04-26'
url = 'https://www.scoresandodds.com/nba?date=2023-06-02'
#url = 'https://www.scoresandodds.com/nba?date=2023-03-07' troubleeeeeee need to fix later, missing game
#url = 'https://www.scoresandodds.com/nba?date=2023-03-27'
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')

In [460]:
%%time
def init_page(url):
    '''
    open url, navigate to menu for BeautifulSoup to access odds
    '''
    # initiate webdriver
    d = webdriver.Chrome('/opt/homebrew/bin/chromedriver')
    d.get(url)

    # isolate and click game_detail button, switch tabs
    game_detail_buttons = d.find_elements_by_xpath("//*[contains(@aria-label, 'Game Details')]")
    for game_detail_button in game_detail_buttons:
        d.execute_script("arguments[0].click();", game_detail_button) 

    # wait for page, and tab from game_detail menu, to load 
    while len(d.find_elements_by_xpath("//*[contains(@id, 'compare-odds-tab')]")) != len(game_detail_buttons):
        pass

    # click compare odds tab
    compare_odds = d.find_elements_by_xpath("//*[contains(@id, 'compare-odds-tab')]")[0]
    d.execute_script("arguments[0].click();", compare_odds) 
    
    return d
    # change bet type
    # bet_type_menu = d.find_elements_by_xpath("//*[contains(@data-content, 'bet-type')]")[0]
    # d.execute_script("arguments[0].click();", bet_type_menu) 

    # while len(d.find_elements_by_xpath("//*[contains(@data-content, '.odds-table-moneyline')]")) == 0:
    #     pass
    # moneyline = d.find_elements_by_xpath("//*[contains(@data-content, '.odds-table-moneyline')]")[0]
    # d.execute_script("arguments[0].click();", moneyline) 

    #.odds-table-total
    
def isolate_rows(page):
    '''
    locate all rows where odds data is kept
    '''
    odds_rows = []
    point_rows = []
    for row in BeautifulSoup(page.page_source, 'html.parser').find_all('tr'):
        if (row.td is not None) and (row.span is not None):
            if 'class' in list(row.td.attrs.keys()):
                if (('game-team' in row.td['class']) or ('game-time' in row.td['class'])
                               ) and ('team-rotation' in row.span['class'][0]):
                    odds_rows.append(row)
        if 'class' in list(row.attrs.keys()):
            try:
                if row['class'][0] == 'event-card-row':
#                     print(row.find_all('a')[0]['aria-label'])
#                     print(row.select('.event-card-score.loss, .event-card-score.win')[0].text.strip())
#                     print(row['data-side'])
                    point_rows.append(row)
            except IndexError:
                pass
    return odds_rows, point_rows

def parse_odds(rows):
    '''
    parse rows for all odds data
    '''
    data = []
    for i in range(int(len(rows)/2)):
        # iterate through pair of rows, every pair is one table, there are 3 tables (spread, total, moneyline)
        first_row = rows[i*2] #away team
        second_row = rows[i*2 + 1] #home team
                
        bet_type = BeautifulSoup(first_row.find_all('td')[1].get('data-content'), 'html.parser').find(
                    'div', {'data-role': 'chassis'}).get('data-market')  # get bet type
        long_date = first_row.find_all('a')[0]['data-value']  # date
        game_name = first_row.find_all('a')[0]['href']  # name
        short_date = first_row.find_all('a')[0].text  # shorter date
        away_team = first_row.find_all('a')[1]['aria-label']  # team name
        away_odds = []
        for odds_index in range(2, 11):
            sportsbook = re.findall('(?<=k\.)\w+(?=\.)|(?<=s\/)\w+', first_row.find_all('a')[odds_index
                                                                                            ]['href'])[0]
            line = re.findall('(?<=\s)\S+(?=\s)', first_row.find_all('a')[odds_index].span.text)[0]
            if (i+1) % 3 == 0: #moneyline bets
                away_odds.append([sportsbook, line, line])
            else: #spread / total bets
                odds = re.findall('(?<=\s)\S+(?=\s)', first_row.find_all('a')[odds_index].small.text)[0]
                away_odds.append([sportsbook, line, odds])

        home_team = second_row.find_all('a')[0]['aria-label']
        home_odds = []
        for odds_index in range(1, 10):
            sportsbook = re.findall('(?<=k\.)\w+(?=\.)|(?<=s\/)\w+', second_row.find_all('a')[odds_index
                                                                                             ]['href'])[0]
            line = re.findall('(?<=\s)\S+(?=\s)', second_row.find_all('a')[odds_index].span.text)[0]
            if (i+1) % 3 == 0: #moneyline bets
                home_odds.append([sportsbook, line, line])
            else: #spread / total bets
                odds = re.findall('(?<=\s)\S+(?=\s)', second_row.find_all('a')[odds_index].small.text)[0]
                home_odds.append([sportsbook, line, odds])

        data.append([bet_type, long_date, away_team, 'away', home_team] + sum(away_odds, []))
        data.append([bet_type, long_date, home_team, 'home', away_team] + sum(home_odds, []))

    return data

def parse_points(rows):
    '''
    parse rows for points data
    calculate spread, total, and which team won
    '''
    data = []
    for i in range(int(len(rows)/2)):
        # iterate through pair of rows, every pair is one table, each pair has two teams from one game
        first_team = rows[i*2] #away team
        second_team = rows[i*2 + 1] #home team
        
        first_team_name = first_team.find_all('a')[0]['aria-label']
        second_team_name = second_team.find_all('a')[0]['aria-label']
        first_team_points = int(first_team.select('.event-card-score.loss, .event-card-score.win')[0
                                                                                ].text.strip())
        second_team_points = int(second_team.select('.event-card-score.loss, .event-card-score.win')[0
                                                                                ].text.strip())
                
        bet_types = ['spread', 'total', 'moneyline']
        for bet_type in bet_types:
            if bet_type == 'spread':
                first_team_points - second_team_points
                data.append(['spread', first_team_name, first_team_points - second_team_points])
                data.append(['spread', second_team_name, second_team_points - first_team_points])
            elif bet_type == 'total':
                total = first_team_points + second_team_points
                data.append(['total', first_team_name, total])
                data.append(['total', second_team_name, total])
            else:
                if first_team_points > second_team_points:
                    data.append(['moneyline', first_team_name, 1])
                    data.append(['moneyline', second_team_name, 0])
                else:
                    data.append(['moneyline', first_team_name, 0])
                    data.append(['moneyline', second_team_name, 1])
        
    return data

def create_results(odds, points):
    '''
    convert odds and points into dataframes
    combine these dataframes to obtain data on if bets hit or not and payout
    add additional columns for info
    '''
    base_columns = ['type', 'datetime', 'team', 'team_home_away', 'opponent']
    side_columns = [item for sublist in [['bookmaker' + str(i), 'line' + str(i), 'odds' + str(i)] for i 
                           in np.arange(1, 10)] for item in sublist]
    odds_df_prelim = pd.DataFrame(odds, columns = base_columns + side_columns)

    data = []
    for i, row in odds_df_prelim.iterrows():
        base_data = list(row[base_columns])
        book_data = list(row[side_columns])
        sub_book_data = [book_data[i:i + 3] for i in range(0, len(book_data), 3)]
        for triplet in sub_book_data:
            data.append(base_data + triplet)
    odds_df = pd.DataFrame(data, columns = base_columns + ['bookmaker', 'line', 'odds'])
    points_df = pd.DataFrame(data = points, columns = ['type', 'team', 'calculated_val'])
    results = odds_df.merge(points_df, left_on = ['type', 'team'], right_on = ['type', 'team'])
    results['line'] = ['+0' if i == 'PK' else i for i in results['line']]
    hits = []
    for i, row in results.iterrows():
        if row['type'] == 'spread':
            if float(row['line']) + row['calculated_val'] > 0:
                hits.append(1)
            elif float(row['line']) + row['calculated_val'] < 0:
                hits.append(0)
            else:
                hits.append('')
        elif row['type'] == 'total':
            if row['line'][0] == 'o': #indicates over
                if row['calculated_val'] > float(row['line'][1:]):
                    hits.append(1)
                elif row['calculated_val'] < float(row['line'][1:]):
                    hits.append(0)
                else:
                    hits.append('')
            if row['line'][0] == 'u': #indicates under:
                if row['calculated_val'] < float(row['line'][1:]):
                    hits.append(1)
                elif row['calculated_val'] > float(row['line'][1:]):
                    hits.append(0)
                else:
                    hits.append('')
        elif row['type'] == 'moneyline':
            hits.append(row['calculated_val'])
    results['hit'] = [0 if i == '' else i for i in hits]
    results['line'] = ['-100' if i == 'even' else i for i in results['line']]
    results['odds'] = [-100 if i == 'even' else int(i) for i in results['odds']]
    results['over_under'] = [i[0] if (i[0] == 'o' or i[0] == 'u') else None for i in results['line']]
    results['line_float'] = [float(i[1:]) if (i[0] == 'o' or i[0] == 'u') else float(i) for i in results['line']]
    results['payout_multiplier'] = results['odds'] * results['hit']
    results['payout_multiplier'] = [100 if i == '' else i for i in results['payout_multiplier']]
    results['payout_multiplier'] = [(abs(i)+100)/abs(i) if i < 0 else i/100+1 if i > 0 else 0 
                                for i in results['payout_multiplier']]
    results['favorite'] = [1 if i[0] == '-' else 0 if i[0] == '+' else None for i in results['line']]
    return results


# finished_page = init_page(url)
# odds_rows, point_rows = isolate_rows(finished_page)
# odds = parse_odds(odds_rows)
# points = parse_points(point_rows)
# results_df = create_results(odds, points)
#finished_page.close()

CPU times: user 17 µs, sys: 86 µs, total: 103 µs
Wall time: 109 µs


In [423]:
def extract_day(date):
    url = f'https://www.scoresandodds.com/nba?date={date}'
    finished_page = init_page(url)
    odds_rows, point_rows = isolate_rows(finished_page)
    odds = parse_odds(odds_rows)
    points = parse_points(point_rows)
    results_df = create_results(odds, points)
    finished_page.close()
    return results_df

In [466]:
season_starts = datetime.strptime("2022-10-18", "%Y-%m-%d")
season_ends = datetime.strptime("2023-04-09", "%Y-%m-%d")
playoff_begins = datetime.strptime("2023-04-11", "%Y-%m-%d")
playoff_ends = datetime.strptime("2023-06-12", "%Y-%m-%d")
date_generated = pd.date_range(begin, end)
dates = list(date_generated.strftime("%Y-%m-%d"))

In [480]:
%%time
succ = []
fail = []
for i in range(len(dates[:50])):
    date = dates[i]
    try:
        extract_day(date)
        print(f'{i + 1}) {date} succeeded.')
        succ.append(date)
    except:
        print(f'{i + 1}) {date} failed.')
        fail.append(date)

1) 2022-10-18 succeeded.
2) 2022-10-19 succeeded.
3) 2022-10-20 succeeded.
4) 2022-10-21 failed.
5) 2022-10-22 failed.
6) 2022-10-23 succeeded.
7) 2022-10-24 succeeded.
8) 2022-10-25 succeeded.
9) 2022-10-26 succeeded.
10) 2022-10-27 succeeded.
11) 2022-10-28 succeeded.
12) 2022-10-29 succeeded.
13) 2022-10-30 succeeded.
14) 2022-10-31 succeeded.
15) 2022-11-01 succeeded.
16) 2022-11-02 succeeded.
17) 2022-11-03 succeeded.
18) 2022-11-04 succeeded.
19) 2022-11-05 succeeded.
20) 2022-11-06 succeeded.
21) 2022-11-07 succeeded.
22) 2022-11-08 failed.
23) 2022-11-09 succeeded.
24) 2022-11-10 succeeded.
25) 2022-11-11 succeeded.
26) 2022-11-12 succeeded.
27) 2022-11-13 succeeded.
28) 2022-11-14 succeeded.
29) 2022-11-15 succeeded.
30) 2022-11-16 failed.
31) 2022-11-17 succeeded.
32) 2022-11-18 failed.
33) 2022-11-19 succeeded.
34) 2022-11-20 failed.
35) 2022-11-21 failed.
36) 2022-11-22 succeeded.
37) 2022-11-23 failed.
38) 2022-11-24 failed.
39) 2022-11-25 failed.
40) 2022-11-26 failed.
41

In [484]:
%%time
succ2 = []
fail2 = []
dates2 = dates[50:100]
for i in range(len(dates2)):
    date = dates2[i]
    try:
        extract_day(date)
        print(f'{i + 1}) {date} succeeded.')
        succ2.append(date)
    except:
        print(f'{i + 1}) {date} failed.')
        fail2.append(date)

1) 2022-12-07 failed.
2) 2022-12-08 succeeded.
3) 2022-12-09 failed.
4) 2022-12-10 succeeded.
5) 2022-12-11 succeeded.
6) 2022-12-12 failed.
7) 2022-12-13 failed.
8) 2022-12-14 failed.
9) 2022-12-15 failed.
10) 2022-12-16 succeeded.
11) 2022-12-17 succeeded.
12) 2022-12-18 succeeded.
13) 2022-12-19 failed.
14) 2022-12-20 failed.
15) 2022-12-21 failed.
16) 2022-12-22 failed.
17) 2022-12-23 failed.
18) 2022-12-24 failed.
19) 2022-12-25 failed.
20) 2022-12-26 failed.
21) 2022-12-27 failed.
22) 2022-12-28 failed.
23) 2022-12-29 succeeded.
24) 2022-12-30 failed.
25) 2022-12-31 succeeded.
26) 2023-01-01 succeeded.
27) 2023-01-02 failed.
28) 2023-01-03 succeeded.
29) 2023-01-04 failed.
30) 2023-01-05 succeeded.
31) 2023-01-06 failed.
32) 2023-01-07 succeeded.
33) 2023-01-08 succeeded.
34) 2023-01-09 failed.
35) 2023-01-10 failed.
36) 2023-01-11 failed.
37) 2023-01-12 succeeded.
38) 2023-01-13 failed.
39) 2023-01-14 succeeded.
40) 2023-01-15 succeeded.
41) 2023-01-16 failed.
42) 2023-01-17 fai